In [1]:
import numpy as np 
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('spam.csv',encoding='latin-1')
df = df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])
train, test = train_test_split(df, test_size=0.3)
df = train 
df.head()

,v1,v2
709,spam,4mths half price Orange line rental & latest c...
2332,ham,We are both fine. Thanks
4365,ham,Mm yes dear look how i am hugging you both. :-P
5551,ham,Wen did you get so spiritual and deep. That's ...
1647,ham,Evening * v good if somewhat event laden. Will...


## Detecting spam messages by Bayes' theorem 😅 ❤️

In [3]:
p = df['v1'].value_counts()
total = p.ham + p.spam
p_of_a = p.ham / total
p_of_not_a = p.spam /total

In [4]:
spam_words = {}
not_spam_words = {}
total_spam_ws = 0
total_not_spam_ws = 0
for row in df.iterrows():
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(row[1]['v2'])
    for word in words:
        if row[1]['v1'] == 'spam':
            spam_words[word] = spam_words.get(word,0) + 1
            total_spam_ws += 1
        else:
            not_spam_words[word] = not_spam_words.get(word,0) + 1
            total_not_spam_ws += 1

In [5]:
def c_prob_of_w(word,isspam):
    try:
        if isspam:
            return spam_words[word]/ total_spam_ws
        return not_spam_words[word] / total_not_spam_ws
    except:
        pass
    return 1/2

In [6]:
def c_prob_of_message(message,isspam):
    r = 1
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(message)
    for word in words:
        r *=  c_prob_of_w(word,isspam)
    return r

In [7]:
def isSpam(message):
    cp_spam = c_prob_of_message(message,True)
    cp_not_spam = c_prob_of_message(message,False)
    lol_spam = p_of_a * cp_spam
    lol_not_spam = p_of_not_a * cp_not_spam
    return lol_spam > lol_not_spam

In [8]:
def t_f(v1):
    if(v1 == 'spam'):
        return True
    return False

In [9]:
total = 0
positive = 0
negative = 0
for row in test.iterrows():
    if t_f(row[1]['v1']) == isSpam(row[1]['v2']):
        positive += 1
    else:
        negative += 1
    total +=1
print((positive / total) * 100)

10.825358851674642


## Exporting the model 🎫 in kaam chalu manner

In [10]:
model = {}
model['spamWords'] = spam_words
model['notSpamWords'] = not_spam_words
model['totalSpamWs'] = total_spam_ws
model['totalNotSpamWs'] = total_not_spam_ws
model['pOfA'] = p_of_a
model['pOfNotA'] = p_of_not_a

In [11]:
# f = open('model.json','wb+')
# f.write(str(model).encode('utf-8'))
# f.close()